**Part.1 Benchmarking and Profiling**


The  Julia  set  is  an  interesting  CPU-bound  problem  for  us  to  begin  with.  It  is  a  fractal sequence that generates a complex output image, named after Gaston Julia.

The Julia set is a mathematical concept visualized as colorful fractal patterns. It's generated by repeatedly applying a mathematical formula to points on the complex plane, determining if they stay bounded or escape to infinity. The resulting shapes are intricate and self-similar, showcasing the beauty of mathematical exploration and iteration.

1.Read the sections ***“Introducing the Julia Set"*** and ***“Calculating the Full Julia Set”*** on Chapter  2.  Profiling  to  Find  Bottlenecksfrom  the book: M.  Gorelick  &  I.  Ozsvald (2020). High Performance Python. Practical Performant Programming for Humans.Second  Edition.  United  States  of  America:  O’Reilly  Media,  Inc.Implement  the chapter functions(Example2-1, 2-2, 2-3 and 2-4)on Python in order to calculate the Julia Set. Make the representation for the false gray and pure gray scale. 

In [404]:
# Example 2-1. Defining global constants for the coordinate space
"""Julia set generator without optional PIL-based image drawing"""
import time

# area of complex space to investigate
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193


In [405]:
# Example 2-3. Our CPU-bound calculation function
def calculate_z_serial_purepython(maxiter, zs, cs):
    '''Calculate output list using Julia update rule'''
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

In [406]:
# Example 2-2. Establishing the coordinate lists as inputs to our calculation function
def calc_pure_python(desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for 1000^2 grid with 300 iterations

In [407]:
# Example 2-4. main for our code
if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with
    # reasonable defaults for a laptop
    calc_pure_python(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 8.3794424533844 seconds


1.  Make the representation for the false gray and pure gray scale. 

In [408]:
# pip install pillow
from PIL import Image
import array


In [409]:
def show_greyscale(output_raw, width, height, max_iterations):
    """Convert list to array, show using PIL"""
    # Convert output to PIL-compatible input
    max_value = float(max(output_raw))
    scale_factor = float(max_iterations)
    scaled = [int(o / scale_factor * 255) for o in output_raw]
    output = array.array('B', scaled)
    # Display with PIL
    im = Image.new("L", (width, height))
    im.frombytes(output.tobytes(), "raw", "L", 0, -1)
    im.show()


In [410]:
def false_grayscale(desired_width, max_iterations):
    """Create false grayscale representation of the Julia set"""
    # Calculate the Julia set
    output = calc_pure_python(desired_width, max_iterations, return_output=True)

    # Find the maximum iteration value for normalization
    max_value = float(max(output))

    # Normalize iteration counts to the 0-255 grayscale range.
    output_raw_limited = [int(float(o) / max_value * 255) for o in output]

    # Convert grayscale values to equivalent RGB values.
    output_rgb = ((o + (256 * o) + (256 ** 2) * o) * 16 for o in output_raw_limited)

    # Convert the RGB generator to an array of unsigned integers ('I').
    output_rgb = array.array('I', output_rgb)

    # Create a new RGB image with the desired dimensions.
    img = Image.new("RGB", (desired_width, desired_width))
    img.frombytes(output_rgb.tobytes(), "raw", "RGBX", 0, -1)
    img.show()


In [411]:
def pure_grayscale(desired_width, max_iterations):
    """Create pure grayscale representation of the Julia set"""
    output = calc_pure_python(desired_width, max_iterations, return_output=True)

    # Maximum iteration count as the scale factor for normalization.
    scale_factor = float(max(output))

    # Normalize the iteration counts to the 0-255 grayscale range.
    scaled = [int(o / scale_factor * 255) for o in output]

    # Convert the scaled iteration counts into an array of unsigned bytes ('B').
    output = array.array('B', scaled)

    # Create a new grayscale image.
    img = Image.new("L", (desired_width, desired_width))
    img.frombytes(output.tobytes(), "raw", "L", 0, -1)
    img.show()

In [412]:
# By adding "draw_output" we allow the function to do the calculationw without the image being generated, also, according to some... it is good practice :)
def calc_pure_python(draw_output, desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    assert sum(output) == 33219980  # this sum is expected for 1000^2 grid with 300 iterations
    if draw_output:
        show_greyscale(output, width, height, max_iterations)

In [413]:
if __name__ == "__main__":
    # Calculate the Julia set using a pure Python solution with reasonable defaults
    calc_pure_python(draw_output=True, desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 8.029278039932251 seconds


2. Define a new function, timefn, which takes a function as an argument: the inner function, measure_time, takes *args (a variable number of positional arguments) and **kwargs (a variable number of key/value arguments) and passes them through to fn for execution. Decorate calculate_z_serial_purepythonwith @timefn to profile it. Implement Example 2-5 and adapt your current source code.

In [414]:
from functools import wraps

In [415]:
# timefn decorator
def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t1 = time.time()
        result = fn(*args, **kwargs)
        t2 = time.time()
        print(f"@timefn: {fn.__name__} took {t2 - t1} seconds")
        return result
    return measure_time

In [416]:
# Function to calculate the Julia set
@timefn
def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

3. Use the timeit modeule to get a coarse measurement of the execution speed of the CPU-bound  function. Runs  10  loops  with  5  repetitions.  Show  how  to do  the measurement on the command lineand on a Jupyter Notebook.

In [417]:
from calc_python import calc_pure_py
%timeit -r 5 -n 10 calc_pure_py(desired_width=1000, max_iterations=300)

Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 9.494616746902466 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.378207683563232 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 8.653819799423218 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.617635250091553 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.516376733779907 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.599990367889404 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.379675626754761 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.24034857749939 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 7.378875017166138 seconds
Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython to

In the command line:

![Timeit Calculation for calc_python.py from the Command Line](timeit_calc_python.jpg)


4. Use the cProfile module to profilethe source code (.py). Sort the results by the time spent inside each function. This will give a view into the slowest parts. Analyze the output and make a syntesis of the findings. Show how to use the cProfile module on the command line and on a Jupyter Notebook.

In [418]:
import time
import cProfile

def calculate_z_serial_purepython(maxiter, zs, cs):
    """Calculate output list using Julia update rule"""
    output = [0] * len(zs)
    for i in range(len(zs)):
        n = 0
        z = zs[i]
        c = cs[i]
        while abs(z) < 2 and n < maxiter:
            z = z * z + c
            n += 1
        output[i] = n
    return output

# Area
x1, x2, y1, y2 = -1.8, 1.8, -1.8, 1.8
c_real, c_imag = -0.62772, -.42193

def calc_pure_py(desired_width, max_iterations):
    """Create a list of complex co-ordinates (zs) and complex parameters (cs), build Julia set and display"""
    x_step = (x2 - x1) / desired_width
    y_step = (y1 - y2) / desired_width
    x = []
    y = []
    ycoord = y2
    while ycoord > y1:
        y.append(ycoord)
        ycoord += y_step
    xcoord = x1
    while xcoord < x2:
        x.append(xcoord)
        xcoord += x_step
    # build a list of co-ordinates and the initial condition for each cell.
    # Note that our initial condition is a constant and could easily be removed,
    # we use it to simulate a real-world scenario with several inputs to our function
    zs = []
    cs = []
    for ycoord in y:
        for xcoord in x:
            zs.append(complex(xcoord, ycoord))
            cs.append(complex(c_real, c_imag))

    print("Length of x:", len(x))
    print("Total elements:", len(zs))
    start_time = time.time()
    output = calculate_z_serial_purepython(max_iterations, zs, cs)
    end_time = time.time()
    secs = end_time - start_time
    print(calculate_z_serial_purepython.__name__ + " took", secs, "seconds")

    #This sum is expected for a 1000^2 grid with 300 iterations
    # It ensures that our code evolves exactly as we'd intended
    assert sum(output) == 33219980

if __name__ == "__main__":
    # Profile the code using cProfile and sort the results by cumulative time (from highest to lowest)
    cProfile.run('calc_pure_py(desired_width=1000, max_iterations=300)', sort='cumulative')


Length of x: 1000
Total elements: 1000000
calculate_z_serial_purepython took 13.248282432556152 seconds
         36222123 function calls in 14.076 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001   14.076   14.076 {built-in method builtins.exec}
        1    0.036    0.036   14.076   14.076 <string>:1(<module>)
        1    0.618    0.618   14.040   14.040 640877485.py:21(calc_pure_py)
        1    9.475    9.475   13.251   13.251 640877485.py:4(calculate_z_serial_purepython)
 34219980    3.776    0.000    3.776    0.000 {built-in method builtins.abs}
  2002000    0.160    0.000    0.160    0.000 {method 'append' of 'list' objects}
        1    0.011    0.011    0.011    0.011 {built-in method builtins.sum}
        3    0.000    0.000    0.001    0.000 {built-in method builtins.print}
       14    0.000    0.000    0.001    0.000 iostream.py:518(write)
       14    0.000    0.000    0.000    0.0

In the command line:

![cProfile Calculation for calc_python.py from the Command Line](cProfile_calc_python.jpg)

5. Use snakeviz to get a high-level understanding of thecPrifile statistics file.Analyze the output and make a syntesis of the findings.

After generating the .prof file with: **python3 -m cProfile -o snake_calc_python.cprof calc_python.py**

And running: **snakeviz snake_calc_python.cprof**

![Snakeviz cProfiling High-level Stats](snakeviz_cprof_1.jpg)
![Snakeviz cProfiling High-level Stats](snakeviz_cprof_2.jpg)

6. Use  the  line_profiler  and  kernprof  file  to  profile  line-by-line  the  function calculate_z_serial_purepython. Analyze  the  output  and  make  a  syntesis  of  the findings.

After running the decorated version of the code within my terminal with: **kernprof -l -v calc_python_line_prof.py** 

![Line_profiler and kernprof line-by-line function](calc_python_line_and_kern.jpg)

7. Use the memory_profiler to diagnose memory usage. Analyze the output and make a syntesis of the findings.

After running: **python3 -m memory_profiler calc_python_memory_profiler.py**

![Memory profiler to diagnose memory usage](calc_python_line_and_kern.jpg)